In this notebook, I'm going to try and rewrite the burstcube class and utility function file to see if I can get things going faster than I orginally did, after all I'm a better programmer now than I was a few months ago. Additionally, there are a few different plots/images I'm hoping to take from this simulation in order to plug on my poster. 


Here's what they are:


A skymap 

A summary of different geometries, and how their loc's differ


Localization avg offset, uncertainty, which is +1 of chi squared for std. deviation. 




# Make sure to update these functions following what they do in this notebook

In [ ]:
#an update to the Sky class. 
from healpy import nside2npix, pix2ang


class Sky():
    """
    Generates an array of GRB's given
    certains strength at different sky positions.
    
    Output should be an array.
    
    The number of pixels used to obtain a higher resolution is correlated to the NSIDE specified. 
    
    
    # of pixels = 12 * nside ^2, also nside has to be a power of 2, but that's besides the point. 
    """
    
    def __init__(self, NSIDE, strength):

        # depending on NSIDE, there will be anywhere
        # from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = nside2npix(NSIDE)

        # want to convert these pixels into theta phi coords.
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(pix2ang(NSIDE, i))


Now I'm looking at the burstcube class, I'll be trying to enhance its structure. 

In [ ]:
"""The following cell contains the "BurstCube" class.  This is the
simulation I hope to use emulate the results of state of the art
simulations on GRB localization, and use these results to characterize
the burstcube spacecraft.

For questions/comments please contact me, Noah Kasmanoff, at
nkasmanoff@gmail.com or https://github.com/nkasmanoff

"""
from sklearn.preprocessing import normalize


import pandas as pd
from numpy import rad2deg, deg2rad, pi, sqrt, add, array, average
from healpy import ang2vec, newvisufunc
import numpy as np

# sometimes one import method works, sometimes another one
# does. Here's a quick fix.
try:
    from BurstCube.NoahSim import burstutils as bf
except ImportError:
    import burstutils as bf

from random import gauss
#import statistics as s


# making classes of objects, allows for different instances of
# burstcube, easy to compare.


class BurstCube():
    #enter initial parameter, such as the tilt and background of the instance. 
    #note that if you want the tilt to be alternating,
    #you need to first make this true, and then enter that value when prompted. 
    def __init__(self, background, dettilt, alternating=False):
        if alternating is False:
            self.tilt = deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = deg2rad(self.tiltB)
            self.tiltC = self.tiltA = deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0, 0]
        self.bg = background
        self.nside = 32
    
    #these properties are each of the detectors initialized in the proper frame. (Spherical)
    
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and
        localize events.  In this software package, they are labelled
        A through D.

        """
        return [self.zenith[0] + self.tiltA, self.zenith[1]]
    
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*pi/2 ]
    
    @property
    def normA(self):
        return  ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 

    #now that the properties of burstucbe have been designed, now its time to test the model's localization capabilities    
    @property
    def initialize(self):  
    
    #first need to include the GRB.
       
        """
        Using x, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        
        talk : boolean
            If desired, prints position by position results. 
        
        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        
        ---
        Create my own GRB loading system here. 
        """
        
        GRB = Sky(self.nside,1)  #inherits GRB
            #range of values used in the fitting. 
        skypoints = len(GRB.sourceangs)   #number of GRBs you're testing
        
        ideal_responses = []
        for i in range(skypoints):  #for each sample 
            
            sourceAng = GRB.sourceangs[i]
#                print("Testing a burst @ " + str(rad2deg(sourceAng)))

            
            sourcexyz = ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst at this position
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            
            """A"""
            sepA=bf.angle(sourcexyz,self.normA)  
            xA = bf.look_up_A(self.normA,sourcexyz)
    
            dtheoryA=GRB.Ao*bf.response(sepA,xA)  
                    
            """B"""
            sepB=bf.angle(sourcexyz,self.normB)
            xB = bf.look_up_B(self.normB,sourcexyz)
            dtheoryB=GRB.Ao*bf.response(sepB,xB)  

            
            """C"""
            sepC=bf.angle(sourcexyz,self.normC)
            xC =  bf.look_up_C(self.normC,sourcexyz)
            dtheoryC=GRB.Ao*bf.response(sepC,xC)  #still need to define strength, brb and gonna do that 

            """D"""
            sepD=bf.angle(sourcexyz,self.normD)
            xD = bf.look_up_D(self.normD,sourcexyz)
            dtheoryD=GRB.Ao*bf.response(sepD,xD)  #still need to define strength, brb and gonna do that 
            ideal_responses.append([dtheoryA,dtheoryB,dtheoryC,dtheoryD])
        ideal_responses = normalize(ideal_responses,axis=1)
        for i in range(len(ideal_responses)):
            #this is a quick fix for removing the normalizing below horizon. 
            if ideal_responses[i][0] == ideal_responses[i][1] == ideal_responses[i][2] == ideal_responses[i][3]:
                ideal_responses[i][0] = 100
                ideal_responses[i][1] = 100
                ideal_responses[i][2] = 100
                ideal_responses[i][3] = 100
        self.ideal_data = pd.DataFrame([])
        self.ideal_data['A'] = ideal_responses[:,0]
        self.ideal_data['B'] = ideal_responses[:,1]
        self.ideal_data['C'] = ideal_responses[:,2]
        self.ideal_data['D'] = ideal_responses[:,3]
        return self.ideal_data

    """Brief interlude..
   
   
   """
    
    def response2GRB(self, GRB,test=False,talk=False):  

    #first need to include the GRB.
       
        """
        Using x, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        
        talk : boolean
            If desired, prints position by position results. 
        
        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        skyvals = []
        if test:
            nsamples = 1
            skypoints = 1

        else:
            #range of values used in the fitting. 
            skypoints = len(GRB.sourceangs)   #number of GRBs you're testing
            nsamples = 100

        actual_responses = []
        for i in range(skypoints):  #for each grb
            
            sourceAng = GRB.sourceangs[i]
            if talk:
                print("Testing bursts @ " + str(rad2deg(sourceAng))+", sampling it " + str(nsamples)+ " times")

            
            sourcexyz = ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst at this position
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            
            for i in range(nsamples): 
                """A"""
                sepA=bf.angle(sourcexyz,self.normA)  
                xA = bf.look_up_A(self.normA,sourcexyz)
    
                dtheoryA=GRB.Ao*bf.response(sepA,xA)  
                    
                countsA = dtheoryA + self.bg
                unccountsA = sqrt(countsA)
                detactualA = gauss(countsA,unccountsA)
                if detactualA-self.bg < 0:
                    detactualA = 0
                detcountsA = detactualA - self.bg
                
                """B"""
                sepB=bf.angle(sourcexyz,self.normB)
                xB = bf.look_up_B(self.normB,sourcexyz)
                dtheoryB=GRB.Ao*bf.response(sepB,xB)  
                countsB = dtheoryB + self.bg 
                unccountsB = sqrt(countsB)
                detactualB = gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = 0
                    
                detcountsB = detactualB - self.bg
                
            
                """C"""
                sepC=bf.angle(sourcexyz,self.normC)
                xC =  bf.look_up_C(self.normC,sourcexyz)
                dtheoryC=GRB.Ao*bf.response(sepC,xC)  #still need to define strength, brb and gonna do that 
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = sqrt(countsC)
                detactualC = gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = 0
                    
                detcountsC = detactualC - self.bg
                
                

                """D"""
                sepD=bf.angle(sourcexyz,self.normD)
                xD = bf.look_up_D(self.normD,sourcexyz)
                dtheoryD=GRB.Ao*bf.response(sepD,xD)  #still need to define strength, brb and gonna do that 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = sqrt(countsD)
                detactualD = gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = 0 
                detcountsD = detactualD - self.bg
                arr = np.array([float(detcountsA),float(detcountsB),float(detcountsC),float(detcountsD)])
                arr = arr.reshape(1,-1)
              #  if talk:
                #    print(arr)
                normalized_arr = normalize(arr,axis=1) #converted 
            #This tab corresponds to a new sky pos being tested, will have to evaluate all of these at once maybe? 
                observed_data = pd.DataFrame([])
                observed_data['A'] = normalized_arr[0][0]* np.ones(len(self.ideal_data))
                observed_data['B'] = normalized_arr[0][1]* np.ones(len(self.ideal_data))
                observed_data['C'] = normalized_arr[0][2]* np.ones(len(self.ideal_data))
                observed_data['D'] = normalized_arr[0][3]* np.ones(len(self.ideal_data))
        
                #SO NOW WITH THIS OBSERVED DATA, COMPARE TO IDEAL RESPONES. 
                chiterms  = (self.ideal_data - observed_data)**2 / self.ideal_data
                observed_data['chisquared'] = chiterms.sum(axis=1)
                chimin = observed_data['chisquared'].loc[observed_data['chisquared'] == min(observed_data['chisquared'])].index[0]
                recpos = pix2ang(ipix=int(chimin),nside=self.nside)
             #   print(recpos)
                recvec = ang2vec(recpos[0],recpos[1])
                locoffset = np.rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                
                locunc.append(locoffset)
                #convert recpos into degrees sepearaiton.
            locunc = np.array(locunc)
           # nanmask = nanmask = np.isnan(locunc)
           # locunc = locunc[~nanmask]
            if talk:
                print("Avg unc: " + str(np.mean(locunc)))
            skyvals.append(np.mean(locunc))
        skyvals = np.array(skyvals)
        return skyvals
    

            
            

Run this one version, see how it goes. 

In [ ]:
NSIDE = 4
STRENGTH = 5000
BACKGROUND = 10
TILT = 45

sim1 = Sky(NSIDE,STRENGTH)

#run this file, and you immediately get
testcube = BurstCube(BACKGROUND,TILT,alternating =False)
_ = testcube.initialize #supress output for now, but it is now a property so we chilling. 

In [ ]:
locoffsets = testcube.response2GRB(sim1,talk=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
im = np.array(locoffsets)
newvisufunc.mollview(im,min=0, max=30,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True)
plt.title('All Sky Localization Accuracy for BurstCube set as ' + str(TILT) + ' deg')  #should add something about design too! 

plt.show()

In [ ]:
len(locoffsets)

In [ ]:
from numpy import nan
x = [1400, 1500, 1600, nan, nan, nan ,1700] #Not in this exact configuration


In [ ]:
nanmask = np.isnan(x)

In [ ]:
x = np.array(x)

In [ ]:
x[~nanmask]

In [ ]:
huh = testcube.ideal_data.values[1616]
shouldbe = testcube.ideal_data.values[0]

In [ ]:
sum((huh - ex) **2 / huh)

In [ ]:
sum((shouldbe - ex) **2 / huh)

In [ ]:
shouldbe[0]**2 + shouldbe[1]**2 + shouldbe[2]**2 + shouldbe[3]**2

In [ ]:
ex[0]**2 + ex[1]**2 + ex[2]**2 + ex[3]**2

In [ ]:
huh

In [ ]:
test_arr = testcube.response2GRB(sim1,test=True,talk=True)

In [ ]:
test_arr

In [ ]:
ideal_arr = testcube.ideal_response2GRB(sim1,talk = False)

So now I have a test array, and the ideal. The next step is to perform a chi square, and see which vals it aligns with. 

In [ ]:
test_arr  #create a d

In [ ]:
def chi_squareds(obs,tru):
    """Calculate the chi squared value of the observed array and the true, the min chi aligns with the best fit. 
    """
    

    #Now for every value in this true array, index corresponds to nside value, which is then the pixel and position. 
    
    

In [ ]:
chi_squareds(test_arr,ideal_arr)

In [ ]:
ideal_arr

In [ ]:
len(ideal_arr)  #also the number of pixels. 

In [ ]:
import pandas as pd
observed_data = pd.DataFrame([])

In [ ]:
observed_data['A'] = test_arr[0][0]* np.ones(len(ideal_arr))
observed_data['B'] = test_arr[0][1]* np.ones(len(ideal_arr))
observed_data['C'] = test_arr[0][2]* np.ones(len(ideal_arr))
observed_data['D'] = test_arr[0][3]* np.ones(len(ideal_arr))

In [ ]:
ideal_data = pd.DataFrame([])
ideal_data['A'] = ideal_arr[:,0]
ideal_data['B'] = ideal_arr[:,1]
ideal_data['C'] = ideal_arr[:,2]
ideal_data['D'] = ideal_arr[:,3]

In [ ]:
ideal_data.head()

In [ ]:
observed_data.head()

(ideal - true) **2 / true 

so for each row, subtract each colum and square

In [ ]:
chiterms  = (ideal_data - observed_data)**2 / observed_data

In [ ]:
chiterms['chisquared'] = chiterms.sum(axis=1)

In [ ]:
chiterms['chisquared'].loc[chiterms['chisquared'] == min(chiterms['chisquared'])].index[0]

In [ ]:
#now convert pixel to coordinate 

In [ ]:
from healpy import pix2ang

np.rad2deg(pix2ang(ipix=50,nside=4))

Where am I leaving off before lunch?


I have an ideal function that prints out the relative # of counts for each detector assuming no noise. 

I have another function that prints out the relative # of counts for each detector assuming noise. 

By using a chi squared at each time this point is sampled, this method allows me to infer the position by looking it up in these two tables, 'observed' and 'idea'. 

Next up is integrating this chi squared and dataframe action into a funciton, and plugging it into a loop. 

The ideal response thing should also be a separate function, otherwise it won't work unless run first. 


Or go like this

1. Initialize object

2. Get responses ... this won't work since of the many such sample points. 
    Possibly average response? But this def loses information so average at the end once its an uncertainty
    
    
3. 
    


So everything that happens here is now what I hope will be what happens over every file, and will be plugged back into the burstcube class and the utility file as well.. 

In [ ]:
from sklearn.preprocessing import normalize

normalized_arr = np.array(normalize(test_arr,axis=0)) #converted 

#quick formating correction
print(normalized_arr)



In [ ]:
normalized_arr = np.array([normalized_arr[0][0],normalized_arr[1][0],normalized_arr[2][0],normalized_arr[3][0]])
#this is lazy coding on my part. It's an extra line but saves me a lott of time going back making this the initial form. 

In [ ]:
normalized_arr.shape

In [ ]:
normalized_arr

Now for this normalized array, perform chi squared fitting. 

It means doing things very differently than I already have. Be prepared for it to get dirty now. Sorry Noah :(

In [ ]:
"""This is where the fitting begings"""
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,A=True)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,B=True)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,C=True)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,D=True)
                
                chisquared = add(add(chiA,chiB),add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = ang2vec(deg2rad(thetaloc),deg2rad(philoc))
                locoffset = rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                
                locunc.append(locoffset)
                loop +=1
            if talk:
                print("Avg loc offset = " + str(average(locunc)) + " deg.")

            self.localizationerrors.append(np.mean(locunc))
        return self.localizationerrors